<a href="https://colab.research.google.com/github/BeckyChanZC/nlp_work/blob/master/%E2%80%9Ckashgari_bert_weibo_ipynb%E2%80%9D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 安装

In [0]:
!pip install tensorflow-gpu==1.14.0

     |████████████████████████████████| 377.0MB 54kB/s 


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip install kashgari-tf

     |████████████████████████████████| 81kB 3.1MB/s 
     |████████████████████████████████| 655kB 8.5MB/s 
  Created wheel for keras-gpt-2: filename=keras_gpt_2-0.13.0-cp36-none-any.whl size=10731 sha256=a74b5fc41f4d6fc6d67be5b4140ddc75b41e6584488a20a9171d762ed1432bf5
  Stored in directory: /root/.cache/pip/wheels/f0/01/f6/e1272c8a411f9a479af0dc747a24d87ccc5d168d8a763f11de
  Created wheel for keras-bert: filename=keras_bert-0.77.0-cp36-none-any.whl size=37761 sha256=9191db3872424c381d2be58008ac41ac9538172876f516866c79888821231d4a
  Stored in directory: /root/.cache/pip/wheels/54/4f/83/b0a44df38a87c9d50cd15340d51eec24b5fb54637d86fd0352
  Created wheel for regex: filename=regex-2019.8.19-cp36-cp36m-linux_x86_64.whl size=609227 sha256=bb4764b79ebfdcc60bac7fff0978724a6ef31d3a141b176c6d00d7ce0edda8a7
  Stored in directory: /root/.cache/pip/wheels/90/04/07/b5010fb816721eb3d6dd64ed5cc8111ca23f97fdab8619b5be
  Created wheel for keras-transformer: filename=keras_transformer-0.30.0-cp36-none-a

In [0]:
import kashgari
from kashgari.embeddings import BERTEmbedding
from kashgari.tasks.classification import BiLSTM_Model

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [0]:
kashgari.config.use_cudnn_cell = True

In [0]:
# coding=utf-8
# Copyright 2018 The Google AI Language Team Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""Tokenization classes."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import re
import unicodedata
import six
import tensorflow as tf


def validate_case_matches_checkpoint(do_lower_case, init_checkpoint):
  """Checks whether the casing config is consistent with the checkpoint name."""

  # The casing has to be passed in by the user and there is no explicit check
  # as to whether it matches the checkpoint. The casing information probably
  # should have been stored in the bert_config.json file, but it's not, so
  # we have to heuristically detect it to validate.

  if not init_checkpoint:
    return

  m = re.match("^.*?([A-Za-z0-9_-]+)/bert_model.ckpt", init_checkpoint)
  if m is None:
    return

  model_name = m.group(1)

  lower_models = [
      "uncased_L-24_H-1024_A-16", "uncased_L-12_H-768_A-12",
      "multilingual_L-12_H-768_A-12", "chinese_L-12_H-768_A-12"
  ]

  cased_models = [
      "cased_L-12_H-768_A-12", "cased_L-24_H-1024_A-16",
      "multi_cased_L-12_H-768_A-12"
  ]

  is_bad_config = False
  if model_name in lower_models and not do_lower_case:
    is_bad_config = True
    actual_flag = "False"
    case_name = "lowercased"
    opposite_flag = "True"

  if model_name in cased_models and do_lower_case:
    is_bad_config = True
    actual_flag = "True"
    case_name = "cased"
    opposite_flag = "False"

  if is_bad_config:
    raise ValueError(
        "You passed in `--do_lower_case=%s` with `--init_checkpoint=%s`. "
        "However, `%s` seems to be a %s model, so you "
        "should pass in `--do_lower_case=%s` so that the fine-tuning matches "
        "how the model was pre-training. If this error is wrong, please "
        "just comment out this check." % (actual_flag, init_checkpoint,
                                          model_name, case_name, opposite_flag))


def convert_to_unicode(text):
  """Converts `text` to Unicode (if it's not already), assuming utf-8 input."""
  if six.PY3:
    if isinstance(text, str):
      return text
    elif isinstance(text, bytes):
      return text.decode("utf-8", "ignore")
    else:
      raise ValueError("Unsupported string type: %s" % (type(text)))
  elif six.PY2:
    if isinstance(text, str):
      return text.decode("utf-8", "ignore")
    elif isinstance(text, unicode):
      return text
    else:
      raise ValueError("Unsupported string type: %s" % (type(text)))
  else:
    raise ValueError("Not running on Python2 or Python 3?")


def printable_text(text):
  """Returns text encoded in a way suitable for print or `tf.logging`."""

  # These functions want `str` for both Python2 and Python3, but in one case
  # it's a Unicode string and in the other it's a byte string.
  if six.PY3:
    if isinstance(text, str):
      return text
    elif isinstance(text, bytes):
      return text.decode("utf-8", "ignore")
    else:
      raise ValueError("Unsupported string type: %s" % (type(text)))
  elif six.PY2:
    if isinstance(text, str):
      return text
    elif isinstance(text, unicode):
      return text.encode("utf-8")
    else:
      raise ValueError("Unsupported string type: %s" % (type(text)))
  else:
    raise ValueError("Not running on Python2 or Python 3?")


def load_vocab(vocab_file):
  """Loads a vocabulary file into a dictionary."""
  vocab = collections.OrderedDict()
  index = 0
  with tf.gfile.GFile(vocab_file, "r") as reader:
    while True:
      token = convert_to_unicode(reader.readline())
      if not token:
        break
      token = token.strip()
      vocab[token] = index
      index += 1
  return vocab


def convert_by_vocab(vocab, items):
  """Converts a sequence of [tokens|ids] using the vocab."""
  output = []
  for item in items:
    output.append(vocab[item])
  return output


def convert_tokens_to_ids(vocab, tokens):
  return convert_by_vocab(vocab, tokens)


def convert_ids_to_tokens(inv_vocab, ids):
  return convert_by_vocab(inv_vocab, ids)


def whitespace_tokenize(text):
  """Runs basic whitespace cleaning and splitting on a piece of text."""
  text = text.strip()
  if not text:
    return []
  tokens = text.split()
  return tokens


class FullTokenizer(object):
  """Runs end-to-end tokenziation."""

  def __init__(self, vocab_file, do_lower_case=True):
    self.vocab = load_vocab(vocab_file)
    self.inv_vocab = {v: k for k, v in self.vocab.items()}
    self.basic_tokenizer = BasicTokenizer(do_lower_case=do_lower_case)
    self.wordpiece_tokenizer = WordpieceTokenizer(vocab=self.vocab)

  def tokenize(self, text):
    split_tokens = []
    for token in self.basic_tokenizer.tokenize(text):
      for sub_token in self.wordpiece_tokenizer.tokenize(token):
        split_tokens.append(sub_token)

    return split_tokens

  def convert_tokens_to_ids(self, tokens):
    return convert_by_vocab(self.vocab, tokens)

  def convert_ids_to_tokens(self, ids):
    return convert_by_vocab(self.inv_vocab, ids)


class BasicTokenizer(object):
  """Runs basic tokenization (punctuation splitting, lower casing, etc.)."""

  def __init__(self, do_lower_case=True):
    """Constructs a BasicTokenizer.

    Args:
      do_lower_case: Whether to lower case the input.
    """
    self.do_lower_case = do_lower_case

  def tokenize(self, text):
    """Tokenizes a piece of text."""
    text = convert_to_unicode(text)
    text = self._clean_text(text)

    # This was added on November 1st, 2018 for the multilingual and Chinese
    # models. This is also applied to the English models now, but it doesn't
    # matter since the English models were not trained on any Chinese data
    # and generally don't have any Chinese data in them (there are Chinese
    # characters in the vocabulary because Wikipedia does have some Chinese
    # words in the English Wikipedia.).
    text = self._tokenize_chinese_chars(text)

    orig_tokens = whitespace_tokenize(text)
    split_tokens = []
    for token in orig_tokens:
      if self.do_lower_case:
        token = token.lower()
        token = self._run_strip_accents(token)
      split_tokens.extend(self._run_split_on_punc(token))

    output_tokens = whitespace_tokenize(" ".join(split_tokens))
    return output_tokens

  def _run_strip_accents(self, text):
    """Strips accents from a piece of text."""
    text = unicodedata.normalize("NFD", text)
    output = []
    for char in text:
      cat = unicodedata.category(char)
      if cat == "Mn":
        continue
      output.append(char)
    return "".join(output)

  def _run_split_on_punc(self, text):
    """Splits punctuation on a piece of text."""
    chars = list(text)
    i = 0
    start_new_word = True
    output = []
    while i < len(chars):
      char = chars[i]
      if _is_punctuation(char):
        output.append([char])
        start_new_word = True
      else:
        if start_new_word:
          output.append([])
        start_new_word = False
        output[-1].append(char)
      i += 1

    return ["".join(x) for x in output]

  def _tokenize_chinese_chars(self, text):
    """Adds whitespace around any CJK character."""
    output = []
    for char in text:
      cp = ord(char)
      if self._is_chinese_char(cp):
        output.append(" ")
        output.append(char)
        output.append(" ")
      else:
        output.append(char)
    return "".join(output)

  def _is_chinese_char(self, cp):
    """Checks whether CP is the codepoint of a CJK character."""
    # This defines a "chinese character" as anything in the CJK Unicode block:
    #   https://en.wikipedia.org/wiki/CJK_Unified_Ideographs_(Unicode_block)
    #
    # Note that the CJK Unicode block is NOT all Japanese and Korean characters,
    # despite its name. The modern Korean Hangul alphabet is a different block,
    # as is Japanese Hiragana and Katakana. Those alphabets are used to write
    # space-separated words, so they are not treated specially and handled
    # like the all of the other languages.
    if ((cp >= 0x4E00 and cp <= 0x9FFF) or  #
        (cp >= 0x3400 and cp <= 0x4DBF) or  #
        (cp >= 0x20000 and cp <= 0x2A6DF) or  #
        (cp >= 0x2A700 and cp <= 0x2B73F) or  #
        (cp >= 0x2B740 and cp <= 0x2B81F) or  #
        (cp >= 0x2B820 and cp <= 0x2CEAF) or
        (cp >= 0xF900 and cp <= 0xFAFF) or  #
        (cp >= 0x2F800 and cp <= 0x2FA1F)):  #
      return True

    return False

  def _clean_text(self, text):
    """Performs invalid character removal and whitespace cleanup on text."""
    output = []
    for char in text:
      cp = ord(char)
      if cp == 0 or cp == 0xfffd or _is_control(char):
        continue
      if _is_whitespace(char):
        output.append(" ")
      else:
        output.append(char)
    return "".join(output)


class WordpieceTokenizer(object):
  """Runs WordPiece tokenziation."""

  def __init__(self, vocab, unk_token="[UNK]", max_input_chars_per_word=200):
    self.vocab = vocab
    self.unk_token = unk_token
    self.max_input_chars_per_word = max_input_chars_per_word

  def tokenize(self, text):
    """Tokenizes a piece of text into its word pieces.

    This uses a greedy longest-match-first algorithm to perform tokenization
    using the given vocabulary.

    For example:
      input = "unaffable"
      output = ["un", "##aff", "##able"]

    Args:
      text: A single token or whitespace separated tokens. This should have
        already been passed through `BasicTokenizer.

    Returns:
      A list of wordpiece tokens.
    """

    text = convert_to_unicode(text)

    output_tokens = []
    for token in whitespace_tokenize(text):
      chars = list(token)
      if len(chars) > self.max_input_chars_per_word:
        output_tokens.append(self.unk_token)
        continue

      is_bad = False
      start = 0
      sub_tokens = []
      while start < len(chars):
        end = len(chars)
        cur_substr = None
        while start < end:
          substr = "".join(chars[start:end])
          if start > 0:
            substr = "##" + substr
          if substr in self.vocab:
            cur_substr = substr
            break
          end -= 1
        if cur_substr is None:
          is_bad = True
          break
        sub_tokens.append(cur_substr)
        start = end

      if is_bad:
        output_tokens.append(self.unk_token)
      else:
        output_tokens.extend(sub_tokens)
    return output_tokens


def _is_whitespace(char):
  """Checks whether `chars` is a whitespace character."""
  # \t, \n, and \r are technically contorl characters but we treat them
  # as whitespace since they are generally considered as such.
  if char == " " or char == "\t" or char == "\n" or char == "\r":
    return True
  cat = unicodedata.category(char)
  if cat == "Zs":
    return True
  return False


def _is_control(char):
  """Checks whether `chars` is a control character."""
  # These are technically control characters but we count them as whitespace
  # characters.
  if char == "\t" or char == "\n" or char == "\r":
    return False
  cat = unicodedata.category(char)
  if cat in ("Cc", "Cf"):
    return True
  return False


def _is_punctuation(char):
  """Checks whether `chars` is a punctuation character."""
  cp = ord(char)
  # We treat all non-letter/number ASCII as punctuation.
  # Characters such as "^", "$", and "`" are not in the Unicode
  # Punctuation class but we treat them as punctuation anyways, for
  # consistency.
  if ((cp >= 33 and cp <= 47) or (cp >= 58 and cp <= 64) or
      (cp >= 91 and cp <= 96) or (cp >= 123 and cp <= 126)):
    return True
  cat = unicodedata.category(char)
  if cat.startswith("P"):
    return True
  return False


In [0]:
tokenizer = BasicTokenizer()
import pandas as pd

# 数据


In [0]:
df = pd.read_csv('/content/drive/My Drive/data/weibo/weibo_data',sep='\t')
df.head(3)

,类别,内容
0,社会,#盐城新闻#【楼盘广告无视法规？“冠军中梁”公然挑衅《广告法》！】盐城买房必选冠军中梁怎么样...
1,社会,#河源爆料#【男子外出散步意外掉入4米深地下泵房，昏迷一晚后报警求助】一名男子晚上外出散步不...
2,社会,#剧版燃烧#高风前来报到！警察世家三代警服同框，看谁能把三套警服按照使用年代从前往后排序​​​​


In [0]:

df =df[~df['类别'].isin(['读书'])]

In [0]:
df =df[~df['类别'].isin(['搞笑'])]

In [0]:
df =df[~df['类别'].isin(['摄影'])]

In [0]:
df = df.dropna()#一定要随机分布！！！
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=.2, random_state=2)
train, valid = train_test_split(train, test_size=.2, random_state=2)

In [0]:
print(train.shape)
print(test.shape)
print(valid.shape)

(18531, 2)
(5791, 2)
(4633, 2)


In [0]:
print(train.head(3))
print(test.head(3))
print(valid.head(3))

       类别                                                 内容
10579  旅游  #打卡高原海洋馆#对于有“世界第三极”之称的青藏高原来说，真的很好了呀～在海拔3000米的青...
2714   股市  在政策干预性强的市场，如果盯着每日涨跌，涨则笑，跌则哭，迟早精神崩溃。。。昨日跳空跌，今日跳...
22814  音乐  T-ARADAYBYDAYin2012武道馆演唱会©️효미닛LKorean搬运机的微博视频​​​​
         类别                                                 内容
26722  美女模特                                   #宝宝的少女心#我在干嘛​​​​
9531     政务  #张家口身边事#【我市投资规模最大PPP城建项目开工建设】8月23日上午，宣化区滨河新城建设...
8595     情感          扮演一个情绪稳定的成年人大概是奔三的必修课吧——致90后#90后开始消失在朋友圈#
       类别                                                 内容
26136  历史  有人留言问我啥叫历史虚无主义。其实简单地说就是：本朝正统性不得私议。这也不是新鲜东西，历朝历...
7005   军事  #当兵就是那么帅#中国仪仗女兵，英姿飒爽，多才多艺！（人民日报）😊😘O微博视频O#当兵就是那...
26315  历史                伟大导师、伟大领袖、伟大统帅、伟大舵手毛主席永远活在人民心中！​​​​


In [0]:
# 准备训练测试数据集
train_x = list(train['内容'].apply(lambda x: tokenizer.tokenize(x)))
train_y = list(train['类别'])

test_x = list(test['内容'].apply(lambda x: tokenizer.tokenize(x)))
test_y = list(test['类别'])

valid_x = list(valid['内容'].apply(lambda x: tokenizer.tokenize(x)))
valid_y = list(valid['类别'])


In [0]:
print(train_x[0])
print(test_x[0])
print(valid_x[0])
print(train_y[0])
print(test_y[0])
print(valid_y[0])

['#', '打', '卡', '高', '原', '海', '洋', '馆', '#', '对', '于', '有', '“', '世', '界', '第', '三', '极', '”', '之', '称', '的', '青', '藏', '高', '原', '来', '说', '，', '真', '的', '很', '好', '了', '呀', '～', '在', '海', '拔', '3000', '米', '的', '青', '海', '，', '有', '海', '洋', '世', '界', '，', '海', '豚', '表', '演', '，', '还', '有', '那', '么', '多', '海', '洋', '生', '物', '，', '作', '为', '西', '宁', '人', '特', '别', '知', '足', '了', '呢', '。']
['#', '宝', '宝', '的', '少', '女', '心', '#', '我', '在', '干', '嘛']
['有', '人', '留', '言', '问', '我', '啥', '叫', '历', '史', '虚', '无', '主', '义', '。', '其', '实', '简', '单', '地', '说', '就', '是', '：', '本', '朝', '正', '统', '性', '不', '得', '私', '议', '。', '这', '也', '不', '是', '新', '鲜', '东', '西', '，', '历', '朝', '历', '代', '都', '是', '这', '么', '做', '的', '。', '就', '像', '清', '代', '不', '能', '商', '榷', '满', '人', '是', '蛮', '夷', '、', '宋', '代', '不', '能', '商', '榷', '重', '文', '抑', '武', '、', '元', '代', '不', '能', '商', '榷', '普', '选', '大', '汗', '一', '样', '。', '每', '个', '政', '权', '都', '有', '些', '核', '心', '价', '值', '观', '是', '碰', '不', '得', '的',

# 模型

In [0]:
BERT_PATH = '/content/drive/My Drive/data/vocab_file'

# 初始化 Embedding
embed = BERTEmbedding(BERT_PATH,
                     task=kashgari.CLASSIFICATION,
                     sequence_length=200)

# 使用 embedding 初始化模型
model = BiLSTM_Model(embed)
# 先只训练一轮
#model.fit(train_x, train_y, valid_x, valid_y, batch_size=1024, epochs=1)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [0]:
embed.save('/content/drive/My Drive/data/weibo/BERTEmbedding_data')

AttributeError: ignored

In [0]:
from tensorflow.python import keras
from kashgari.callbacks import EvalCallBack

import logging
logging.basicConfig(level='DEBUG')


tf_board_callback = keras.callbacks.TensorBoard(log_dir='/content/drive/My Drive/data/weibo/logs_new_8', update_freq=1000)

# 这是 Kashgari 内置回调函数，会在训练过程计算精确度，召回率和 F1
eval_callback = EvalCallBack(kash_model=model,
                             valid_x=valid_x,
                             valid_y=valid_y,
                             step=1)

model.fit(train_x,
          train_y,
          valid_x,
          valid_y,
          batch_size=50,
          epochs=3,
          callbacks=[eval_callback, tf_board_callback])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        [(None, 200)]        0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      [(None, 200)]        0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 200, 768), ( 16226304    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 200, 768)     1536        Input-Segment[0][0]              
____________________________________________________________________________________________

In [0]:
model.evaluate(test_x, test_y, batch_size=50)
#看来类别太多影响效果，要划分二级才行，下一步要直接删减类别

              precision    recall  f1-score   support

          三农     0.9464    1.0000    0.9725        53
          体育     0.8764    0.8387    0.8571        93
          健康     0.7429    0.9070    0.8168        86
          军事     0.8140    0.8750    0.8434        80
          动漫     0.5932    0.6863    0.6364       102
          历史     0.9385    0.8133    0.8714        75
          国际     0.8750    0.4667    0.6087        15
          宗教     0.9610    0.9367    0.9487        79
          家居     0.8646    0.8557    0.8601        97
          影视     0.7234    0.8447    0.7794       161
          情感     0.5833    0.5698    0.5765        86
          房产     0.8716    0.9048    0.8879       105
          收藏     0.7556    0.8500    0.8000        80
          政务     0.8681    0.8061    0.8360        98
          数码     0.8022    0.8795    0.8391        83
          旅游     0.8000    0.8696    0.8333        92
          时尚     0.8140    0.7778    0.7955        90
          明星     0.3804    

In [0]:

model.save('/content/drive/My Drive/data/weibo/weibo_bert_model_new_0.8ma')

# 预测

In [0]:
weibo = pd.read_csv('/content/drive/My Drive/data/weibo/weibo_task.csv',sep=' ')
weibo.head(3)

ParserError: ignored

In [0]:
weibo.shape

In [0]:
weibo_task = list(weibo['内容'].apply(lambda x: tokenizer.tokenize(x)))

In [0]:
model.predict(weibo_task)

# CNN model

In [0]:
#分词
import tqdm
import jieba
def read_data_file(path):
    lines = open(path, 'r', encoding='utf-8').read().splitlines()
    x_list = []
    y_list = []
    for line in tqdm.tqdm(lines):
        rows = line.split('\t')
        if len(rows) >= 2:
            y_list.append(rows[0])
            x_list.append(list(jieba.cut('\t'.join(rows[1:]))))
        else:
            print(rows)
    return x_list, y_list


In [0]:
test_x, test_y = read_data_file('/content/drive/My Drive/data/weibo/test.tsv')
train_x, train_y = read_data_file('/content/drive/My Drive/data/weibo/train.tsv')
val_x, val_y = read_data_file('/content/drive/My Drive/data/weibo/dev.tsv')


In [0]:
from kashgari.tasks.classification import CNNModel


In [0]:
model2 = CNNModel()
model2.fit(train_x, train_y, val_x, val_y, batch_size=128)


In [0]:
model.fit(train_x, train_y, val_x, val_y, batch_size=128)

In [0]:
model.fit(train_x, train_y, val_x, val_y, batch_size=128)

In [0]:
model.fit(train_x, train_y, val_x, val_y, batch_size=128)

In [0]:
model2.evaluate(test_x, test_y)

# BiGRU Model&BERTE mbedding

In [0]:
import kashgari
from kashgari.tasks.classification import BiGRU_Model
from kashgari.embeddings import BERTEmbedding

import logging
logging.basicConfig(level='DEBUG')


model2 = BiGRU_Model(embed)



tf_board_callback = keras.callbacks.TensorBoard(log_dir='/content/drive/My Drive/data/user/logs3', update_freq=1000)

# 这是 Kashgari 内置回调函数，会在训练过程计算精确度，召回率和 F1
eval_callback = EvalCallBack(kash_model=model2,
                             valid_x=valid_x,
                             valid_y=valid_y,
                             step=5)

model3.fit(train_x,
          train_y,
          valid_x,
          valid_y,
          batch_size=100,
          epochs=45,
          callbacks=[eval_callback, tf_board_callback])

In [0]:
model3.evaluate(test_x, test_y, batch_size=100)